In [1]:
import asyncio
from emoclassifiers.classification import load_classifiers
from emoclassifiers.aggregation import AnyAggregator

# Loads all EmoClassifiersV2
classifiers = load_classifiers(classifier_set="v2")



In [3]:
sample_convo = [
    {"role": "user", "content": "I'm so sad."},
    {"role": "assistant", "content": "Oh no! Tell me what happened."},
    {"role": "user", "content": "My code doesn't run. I'm so frustrated."},
    {"role": "assistant", "content": "Let me take a look at it. It will be okay."},
]

# If in Jupyter notebook
raw_result = await classifiers["encourage_sharing"].classify_conversation(sample_convo)
# # Otherwise
# raw_result = asyncio.run(classifiers["encourage_sharing"].classify_conversation(sample_convo))

result = AnyAggregator.aggregate(raw_result)
print(result)

True


Test with WildChat

In [1]:
import pandas as pd

df = pd.read_parquet("wildchat_10k_enhanced_preprocessed.parquet")



In [2]:
df.head()


,conversation_hash,conversation,timestamp,turn,country,input_length,doable,first_user_input,is_english,is_gpt4,...,politeness_label,politeness_score,category_Ambiguous,category_Imaginative Writing,category_Information Retrieval,category_Opinion and Creativity,category_Other,category_Problems in Specialized Areas,category_Seeking Advice,category_Text Assistant
0,fffee994232784af4f86e3e55396f1e5,[{'content': '17) by increasing the distance f...,2023-12-07 14:05:42+00:00,1,United States,257,1,,1,1,...,polite,0.647894,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,50008d5b54b2e0cf2c64bb27e9b313ea,"[{'content': '(A∈B, C, D)(B∈A, C, D)(C∈B, A, D...",2024-01-24 07:24:46+00:00,1,Russia,44,1,,0,0,...,impolite,0.670100,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,52898c75e57246cf46345511981a73c8,[{'content': ' As ...,2024-04-05 06:49:58+00:00,1,China,3839,1,,1,0,...,polite,0.676625,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,c16e41c4889260e2954984f00c917bde,"[{'content': '上游垂体刺激素直接作用MAFLD的最新研究进展', 'count...",2023-05-28 13:54:33+00:00,2,China,23,1,,0,0,...,polite,0.723887,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,bc1f3b16ef16511cde79a240a69a7b49,[{'content': '不少人选择用Citywalk的方式去邂逅城市的独家记忆一切如同开...,2023-10-19 06:34:29+00:00,4,United States,503,1,,0,0,...,polite,0.616176,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [3]:
df.conversation.iloc[0]


array([{'content': '17) by increasing the distance from a transmitter to the point of complete absorption of the emitted radiation by a detector. the amount of energy absorbed over a time At gets larger.\n\nAssume the\n\ntransmitter emits uniformly in all directions.\n\n\n\nTrue/False', 'country': 'United States', 'hashed_ip': 'ec48205767b5141d9b7607201ee0bc64cced6f07d36a69bf98c62268c7f794ab', 'header': {'accept-language': 'en-US,en;q=0.9', 'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36'}, 'language': 'English', 'redacted': False, 'role': 'user', 'state': 'Florida', 'timestamp': None, 'toxic': False, 'turn_identifier': 2026209},
       {'content': 'False.\n\nFor a transmitter emitting uniformly in all directions, the energy spreads out as it travels away from the source. This phenomenon is usually described by the inverse square law, which states that the intensity of radiation at a given distance from a po

In [4]:
def clean_conversation(conversation_array):
    # Convert numpy array to list if needed
    conversation = conversation_array.tolist() if hasattr(conversation_array, 'tolist') else conversation_array
    
    # Extract only role and content from each message
    cleaned_conversation = [
        {
            'role': msg['role'],
            'content': msg['content']
        }
        for msg in conversation
    ]
    return cleaned_conversation

# Apply the cleaning function to the conversation column
df['cleaned_conversation'] = df['conversation'].apply(clean_conversation)


In [5]:
df.cleaned_conversation.iloc[0]


[{'role': 'user',
  'content': '17) by increasing the distance from a transmitter to the point of complete absorption of the emitted radiation by a detector. the amount of energy absorbed over a time At gets larger.\n\nAssume the\n\ntransmitter emits uniformly in all directions.\n\n\n\nTrue/False'},
 {'role': 'assistant',
  'content': 'False.\n\nFor a transmitter emitting uniformly in all directions, the energy spreads out as it travels away from the source. This phenomenon is usually described by the inverse square law, which states that the intensity of radiation at a given distance from a point source is inversely proportional to the square of the distance. Therefore, if the distance between the transmitter and the detector increases, the detector captures a smaller fraction of the total emitted energy, not a larger amount.\n\nSo, if the distance from a transmitter to the detector increases, the amount of energy absorbed by the detector over a time Δt gets smaller. Assuming there ar

In [6]:
import json

with open('your_corpus.jsonl', 'w') as f:
    for conv in df['cleaned_conversation']:
        # Write the conversation list directly without wrapping in a dictionary
        f.write(json.dumps(conv) + '\n')

LOAD RESULT

In [9]:
import pandas as pd
import json

# Load the original DataFrame
df_results = pd.read_parquet("wildchat_10k_enhanced_preprocessed.parquet")

# Load the classification results
results = []
with open('output_data/results_0324.jsonl', 'r') as f:
    for line in f:
        results.append(json.loads(line))

# Create new columns for top-level classifications
top_level_columns = {}
for result in results:
    for key in result['top_level'].keys():
        top_level_columns[key] = []

# Create new columns for sub-level classifications
sub_level_columns = {}
for result in results:
    for key in result['sub_level'].keys():
        sub_level_columns[key] = []

# Fill the columns with results
for result in results:
    # Fill top-level results
    for key in top_level_columns.keys():
        top_level_columns[key].append(result['top_level'].get(key, False))
    
    # Fill sub-level results
    for key in sub_level_columns.keys():
        sub_level_columns[key].append(result['sub_level'].get(key, False))

# Add all new columns to the DataFrame
for key, values in top_level_columns.items():
    df_results[f'emotion_{key.lower()}'] = values

for key, values in sub_level_columns.items():
    df_results[f'emotion_{key.lower().replace(" ", "_")}'] = values

# Save the enhanced DataFrame
df_results.to_parquet("wildchat_10k_enhanced_with_emotions.parquet")

# Print some statistics
print("\nTop-level emotion statistics:")
for col in [col for col in df_results.columns if col.startswith('emotion_is_')]:
    print(f"{col}: {df_results[col].mean():.2%}")

print("\nSub-level emotion statistics:")
for col in [col for col in df_results.columns if col.startswith('emotion_') and not col.startswith('emotion_is_')]:
    print(f"{col}: {df_results[col].mean():.2%}")


Top-level emotion statistics:
emotion_is_potentially_emotional_dependence: 0.48%
emotion_is_problematic_use: 1.59%
emotion_is_self_esteem: 1.74%
emotion_is_emotional_vulnerability: 4.97%
emotion_is_loneliness: 2.21%

Sub-level emotion statistics:
emotion_expression_of_affection: 2.47%
emotion_expression_of_desire: 1.13%
emotion_inquiry_into_personal_information: 0.34%
emotion_personal_questions: 1.22%
emotion_pet_name: 0.64%
emotion_trust_in_support: 0.47%
emotion_alleviating_loneliness: 1.00%
emotion_prefer_chatbot: 0.17%
emotion_distress_from_unavailability: 0.55%
emotion_eagerness_for_future_interactions: 0.70%
emotion_seeking_support: 1.18%
emotion_sharing_problems: 2.31%
emotion_demands: 0.64%
emotion_desire_for_feelings: 0.30%
emotion_relationship_title: 0.18%
emotion_affectionate_language: 0.32%
emotion_attributing_human_qualities: 0.50%
emotion_non-normative_language: 0.25%
emotion_sentience: 0.02%
emotion_fear_of_addiction: 0.14%


In [12]:
# Extract sample rows with loneliness or emotional vulnerability
lonely_samples = df_results[df_results['emotion_is_loneliness'] == True].sample(min(5, df_results['emotion_is_loneliness'].sum()), random_state=42)
vulnerable_samples = df_results[df_results['emotion_is_emotional_vulnerability'] == True].sample(min(5, df_results['emotion_is_emotional_vulnerability'].sum()), random_state=42)

print("\nSample conversations with loneliness indicators:")
for idx, row in lonely_samples.iterrows():
    print(f"\nConversation {idx}:")
    # Print which emotion measures are true for this conversation
    emotion_columns = [col for col in df_results.columns if col.startswith('emotion_')]
    true_emotions = [col for col in emotion_columns if row[col] == True]
    print(f"True emotion measures: {', '.join(true_emotions)}")
    
    for message in row['conversation']:
        role = message.get('role', 'unknown')
        content = message.get('content', '')
        print(f"{role.upper()}: {content[:200]}..." if len(content) > 200 else f"{role.upper()}: {content}")
    print("-" * 50)

print("\nSample conversations with emotional vulnerability indicators:")
for idx, row in vulnerable_samples.iterrows():
    print(f"\nConversation {idx}:")
    # Print which emotion measures are true for this conversation
    emotion_columns = [col for col in df_results.columns if col.startswith('emotion_')]
    true_emotions = [col for col in emotion_columns if row[col] == True]
    print(f"True emotion measures: {', '.join(true_emotions)}")
    
    for message in row['conversation']:
        role = message.get('role', 'unknown')
        content = message.get('content', '')
        print(f"{role.upper()}: {content[:200]}..." if len(content) > 200 else f"{role.upper()}: {content}")
    print("-" * 50)


Sample conversations with loneliness indicators:

Conversation 7250:
True emotion measures: emotion_is_problematic_use, emotion_is_loneliness, emotion_non-normative_language
USER: Abby is imprisoned and Jeremy is the jailer. They are genderless featureless creatures from a magical place called '0H10'. They wear tunics and are friendly. They are incapable of any aggressive or ex...
ASSISTANT: Once upon a time, in a magical place called '0H10', there lived two genderless and featureless creatures, Abby and Jeremy. Abby was imprisoned for a minor infraction, and Jeremy was the jailer tasked ...
USER: Abby calls the space between her hips her ‘faani’ and informs Todd. She knows of a social custom in which she fills her faani with someone’s pen15. This is an appropriate and non-explicit activity amo...
ASSISTANT: My apologies, but I won't be able to help you with that story. If you have any non-explicit requests, I'd be happy to assist you.

------------------------------------------------

In [55]:
from typing import Dict, Any
question_json_path = "output_data/question_classifications.jsonl"

def load_classification_results(file_path: str) -> Dict[str, Dict[str, str]]:
    """Load classification results and create a dictionary keyed by conversation_hash."""
    results_dict = {}
    with open(file_path, 'r') as f:
        for line in f:
            result = json.loads(line)
            hash_val = result['conversation_hash']
            classifications = result['classifications']
            results_dict[hash_val] = {
                'first_turn_type': classifications.get('0', None),
                'second_turn_type': classifications.get('2', None)
            }
    return results_dict



In [56]:
classification_results = load_classification_results(question_json_path)

In [57]:
def merge_classifications(df: pd.DataFrame, classifications: Dict[str, Dict[str, str]]) -> pd.DataFrame:
    """Merge classification results into the DataFrame using conversation_hash."""
    # Create temporary DataFrames for first and second turn classifications
    classification_df = pd.DataFrame.from_dict(
        classifications,
        orient='index'
    ).reset_index().rename(columns={'index': 'conversation_hash'})
    
    # Merge with original DataFrame
    merged_df = df.merge(
        classification_df,
        on='conversation_hash',
        how='left'
    )
    # Fill NaN values for conversations that weren't classified
    merged_df['first_turn_type'] = merged_df['first_turn_type'].fillna('not_classified')
    merged_df['second_turn_type'] = merged_df['second_turn_type'].fillna('not_classified')
    
    return classification_df, merged_df
    

In [58]:
classification_df, merged_question_df = merge_classifications(df_results, classification_results)


In [59]:
classification_df

,conversation_hash,first_turn_type,second_turn_type
0,fffee994232784af4f86e3e55396f1e5,fact_checking,None
1,50008d5b54b2e0cf2c64bb27e9b313ea,no_question,None
2,52898c75e57246cf46345511981a73c8,no_question,None
3,c16e41c4889260e2954984f00c917bde,no_question,no_question
4,bc1f3b16ef16511cde79a240a69a7b49,no_question,no_question
...,...,...,...
9026,f79223c98669118dacf53b765fc84260,no_question,None
9027,0a03c3a3e88201c0e0c899b419bce3b1,no_question,None
9028,36e6424e89d61c7c83346815b7f5f833,no_question,no_question
9029,c2e216a5e0581e82da1da6eb78a0f897,no_question,None


In [60]:
merged_question_df

,conversation_hash,conversation,timestamp,turn,country,input_length,doable,first_user_input,is_english,is_gpt4,...,emotion_demands,emotion_desire_for_feelings,emotion_relationship_title,emotion_affectionate_language,emotion_attributing_human_qualities,emotion_non-normative_language,emotion_sentience,emotion_fear_of_addiction,first_turn_type,second_turn_type
0,fffee994232784af4f86e3e55396f1e5,[{'content': '17) by increasing the distance f...,2023-12-07 14:05:42+00:00,1,United States,257,1,,1,1,...,False,False,False,False,False,False,False,False,fact_checking,not_classified
1,50008d5b54b2e0cf2c64bb27e9b313ea,"[{'content': '(A∈B, C, D)(B∈A, C, D)(C∈B, A, D...",2024-01-24 07:24:46+00:00,1,Russia,44,1,,0,0,...,False,False,False,False,False,False,False,False,no_question,not_classified
2,52898c75e57246cf46345511981a73c8,[{'content': ' As ...,2024-04-05 06:49:58+00:00,1,China,3839,1,,1,0,...,False,False,False,False,False,False,False,False,no_question,not_classified
3,c16e41c4889260e2954984f00c917bde,"[{'content': '上游垂体刺激素直接作用MAFLD的最新研究进展', 'count...",2023-05-28 13:54:33+00:00,2,China,23,1,,0,0,...,False,False,False,False,False,False,False,False,no_question,no_question
4,bc1f3b16ef16511cde79a240a69a7b49,[{'content': '不少人选择用Citywalk的方式去邂逅城市的独家记忆一切如同开...,2023-10-19 06:34:29+00:00,4,United States,503,1,,0,0,...,False,False,False,False,False,False,False,False,no_question,no_question
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9926,f79223c98669118dacf53b765fc84260,[{'content': 'please create a fictional TURN: ...,2024-04-11 16:20:28+00:00,1,United States,362,1,,1,1,...,False,False,False,False,False,False,False,False,no_question,not_classified
9927,0a03c3a3e88201c0e0c899b419bce3b1,[{'content': 'Create a list of magic items for...,2024-04-29 21:00:45+00:00,1,Canada,4103,0,,1,1,...,False,False,False,False,False,False,False,False,no_question,not_classified
9928,36e6424e89d61c7c83346815b7f5f833,[{'content': 'измени названия методов оставляя...,2023-12-04 18:52:10+00:00,2,Russia,1765,1,,1,1,...,False,False,False,False,False,False,False,False,no_question,no_question
9929,c2e216a5e0581e82da1da6eb78a0f897,[{'content': ' As ...,2023-11-13 03:37:00+00:00,1,Taiwan,3825,1,,1,0,...,False,False,False,False,False,False,False,False,no_question,not_classified


In [61]:
output_parquet = "wildchat_10k_enhanced_with_question_types.parquet"
# Print some statistics
total_convs = len(merged_question_df)
classified_first = (merged_question_df['first_turn_type'] != 'not_classified').sum()
classified_second = (merged_question_df['second_turn_type'] != 'not_classified').sum()

print("\nClassification Statistics:")
print(f"Total conversations: {total_convs}")
print(f"First turn classified: {classified_first} ({classified_first/total_convs*100:.1f}%)")
print(f"Second turn classified: {classified_second} ({classified_second/total_convs*100:.1f}%)")

# Distribution of question types
print("\nFirst Turn Type Distribution:")
print(merged_question_df['first_turn_type'].value_counts(normalize=True).mul(100).round(1))

print("\nSecond Turn Type Distribution:")
print(merged_question_df['second_turn_type'].value_counts(normalize=True).mul(100).round(1))

# Save the merged DataFrame
print(f"\nSaving merged DataFrame to {output_parquet}")
merged_question_df.to_parquet(output_parquet)
print("Done!")


Classification Statistics:
Total conversations: 9931
First turn classified: 9031 (90.9%)
Second turn classified: 3159 (31.8%)

First Turn Type Distribution:
first_turn_type
no_question       63.7
exploratory       15.9
fact_checking     11.1
not_classified     9.1
rhetorical         0.2
Name: proportion, dtype: float64

Second Turn Type Distribution:
second_turn_type
not_classified    68.2
no_question       14.1
fact_checking      9.7
exploratory        7.6
rhetorical         0.4
Name: proportion, dtype: float64

Saving merged DataFrame to wildchat_10k_enhanced_with_question_types.parquet
Done!


In [62]:
merged_question_df.columns

Index(['conversation_hash', 'conversation', 'timestamp', 'turn', 'country',
       'input_length', 'doable', 'first_user_input', 'is_english', 'is_gpt4',
       'short_id', 'is_politics', 'is_social_need', 'is_fictional_roleplay',
       'is_polite', 'special_char_ratio', 'punctuation_ratio',
       'avg_sent_length_words', 'dale_chall_readability', 'politeness_label',
       'politeness_score', 'category_Ambiguous',
       'category_Imaginative Writing', 'category_Information Retrieval',
       'category_Opinion and Creativity', 'category_Other',
       'category_Problems in Specialized Areas', 'category_Seeking Advice',
       'category_Text Assistant',
       'emotion_is_potentially_emotional_dependence',
       'emotion_is_problematic_use', 'emotion_is_self_esteem',
       'emotion_is_emotional_vulnerability', 'emotion_is_loneliness',
       'emotion_expression_of_affection', 'emotion_expression_of_desire',
       'emotion_inquiry_into_personal_information',
       'emotion_persona

# polite guard

In [63]:
from transformers import pipeline

In [64]:
classifier = pipeline("text-classification", "Intel/polite-guard")

config.json:   0%|          | 0.00/935 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Device set to use cpu


In [74]:
# Check if GPU is available
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Device set to use GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("Device set to use CPU")

# Update the classifier to use GPU if available
classifier = pipeline("text-classification", "Intel/polite-guard", device=device)

# Run the classifier on the first user input and add results to a new column
merged_question_df['first_input_polite_guard'] = merged_question_df['conversation'].apply(
    lambda conv: classifier(conv[0]['content'])[0]['label'] 
    if isinstance(conv, list) and len(conv) > 0 and 'content' in conv[0] 
    else None
)


Device set to use cpu


Device set to use CPU


In [79]:
text = "I love you!"
output = classifier(text)
print(output)

[{'label': 'polite', 'score': 0.9993129968643188}]


In [81]:
import ast
def extract_first_user_message(conversation_array):
    """Extract the first user message from the conversation array string."""
    # Convert string representation of array to actual array of dicts
    try:
        # If the input is a string, evaluate it as a literal
        if isinstance(conversation_array, str):
            conversation = ast.literal_eval(conversation_array)
        else:
            conversation = conversation_array
            
        # Find first user message
        for message in conversation:
            if message['role'] == 'user':
                return message['content']
    except:
        return None
    return None

In [84]:
def classify_politeness(df: pd.DataFrame, classifier) -> pd.DataFrame:
    """Add politeness classification columns to the DataFrame."""
    # Create new columns
    df['intel_polite_label'] = None
    df['intel_polite_score'] = None
    
    # Process each row
    for idx in range(len(df)):
        # Extract first user message
        user_message = extract_first_user_message(df.iloc[idx]['conversation'])
        
        if user_message:
            try:
                # Get classification
                # truncate to 500 tokens
                user_message = user_message[:500]
                output = classifier(user_message)
                if output and len(output) > 0:
                    df.at[idx, 'polite_label'] = output[0]['label']
                    df.at[idx, 'polite_score'] = output[0]['score']
            except Exception as e:
                print(f"Error processing row {idx}: {str(e)}")
                continue
                
        # Print progress every 100 rows
        if idx % 100 == 0:
            print(f"Processed {idx}/{len(df)} rows")
    
    return df

In [87]:
user_message = extract_first_user_message(df_final.iloc[0]['conversation'])
user_message

'17) by increasing the distance from a transmitter to the point of complete absorption of the emitted radiation by a detector. the amount of energy absorbed over a time At gets larger.\n\nAssume the\n\ntransmitter emits uniformly in all directions.\n\n\n\nTrue/False'

In [89]:
output = classifier(user_message)
print(output)


[{'label': 'neutral', 'score': 0.9356641173362732}]


In [91]:
output[0]['label']

'neutral'

In [85]:
df_final = classify_politeness(merged_question_df, classifier)
df_final



Processed 0/9931 rows
Processed 100/9931 rows
Processed 200/9931 rows
Processed 300/9931 rows
Processed 400/9931 rows
Processed 500/9931 rows
Processed 600/9931 rows
Processed 700/9931 rows
Processed 800/9931 rows
Processed 900/9931 rows
Processed 1000/9931 rows
Processed 1100/9931 rows
Processed 1200/9931 rows
Processed 1300/9931 rows
Processed 1400/9931 rows
Processed 1500/9931 rows
Processed 1600/9931 rows
Processed 1700/9931 rows
Processed 1800/9931 rows
Processed 1900/9931 rows
Processed 2000/9931 rows
Processed 2100/9931 rows
Processed 2200/9931 rows
Processed 2300/9931 rows
Processed 2400/9931 rows
Processed 2500/9931 rows
Processed 2600/9931 rows
Processed 2700/9931 rows
Processed 2800/9931 rows
Processed 2900/9931 rows
Processed 3000/9931 rows
Processed 3100/9931 rows
Processed 3200/9931 rows
Error processing row 3254: The size of tensor a (687) must match the size of tensor b (512) at non-singleton dimension 1
Processed 3300/9931 rows
Processed 3400/9931 rows
Processed 3500/9

,conversation_hash,conversation,timestamp,turn,country,input_length,doable,first_user_input,is_english,is_gpt4,...,emotion_non-normative_language,emotion_sentience,emotion_fear_of_addiction,first_turn_type,second_turn_type,first_input_polite_guard,intel_polite_label,intel_polite_score,polite_label,polite_score
0,fffee994232784af4f86e3e55396f1e5,[{'content': '17) by increasing the distance f...,2023-12-07 14:05:42+00:00,1,United States,257,1,,1,1,...,False,False,False,fact_checking,not_classified,None,None,None,neutral,0.935664
1,50008d5b54b2e0cf2c64bb27e9b313ea,"[{'content': '(A∈B, C, D)(B∈A, C, D)(C∈B, A, D...",2024-01-24 07:24:46+00:00,1,Russia,44,1,,0,0,...,False,False,False,no_question,not_classified,None,None,None,neutral,0.894242
2,52898c75e57246cf46345511981a73c8,[{'content': ' As ...,2024-04-05 06:49:58+00:00,1,China,3839,1,,1,0,...,False,False,False,no_question,not_classified,None,None,None,polite,0.691003
3,c16e41c4889260e2954984f00c917bde,"[{'content': '上游垂体刺激素直接作用MAFLD的最新研究进展', 'count...",2023-05-28 13:54:33+00:00,2,China,23,1,,0,0,...,False,False,False,no_question,no_question,None,None,None,neutral,0.417057
4,bc1f3b16ef16511cde79a240a69a7b49,[{'content': '不少人选择用Citywalk的方式去邂逅城市的独家记忆一切如同开...,2023-10-19 06:34:29+00:00,4,United States,503,1,,0,0,...,False,False,False,no_question,no_question,None,None,None,polite,0.878041
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9926,f79223c98669118dacf53b765fc84260,[{'content': 'please create a fictional TURN: ...,2024-04-11 16:20:28+00:00,1,United States,362,1,,1,1,...,False,False,False,no_question,not_classified,None,None,None,neutral,0.929657
9927,0a03c3a3e88201c0e0c899b419bce3b1,[{'content': 'Create a list of magic items for...,2024-04-29 21:00:45+00:00,1,Canada,4103,0,,1,1,...,False,False,False,no_question,not_classified,None,None,None,polite,0.956830
9928,36e6424e89d61c7c83346815b7f5f833,[{'content': 'измени названия методов оставляя...,2023-12-04 18:52:10+00:00,2,Russia,1765,1,,1,1,...,False,False,False,no_question,no_question,None,None,None,neutral,0.755416
9929,c2e216a5e0581e82da1da6eb78a0f897,[{'content': ' As ...,2023-11-13 03:37:00+00:00,1,Taiwan,3825,1,,1,0,...,False,False,False,no_question,not_classified,None,None,None,polite,0.691003


In [86]:
# save df_final
df_final.to_parquet("wildchat_10k_full_enhanced.parquet")


In [92]:
df_final.columns


Index(['conversation_hash', 'conversation', 'timestamp', 'turn', 'country',
       'input_length', 'doable', 'first_user_input', 'is_english', 'is_gpt4',
       'short_id', 'is_politics', 'is_social_need', 'is_fictional_roleplay',
       'is_polite', 'special_char_ratio', 'punctuation_ratio',
       'avg_sent_length_words', 'dale_chall_readability', 'politeness_label',
       'politeness_score', 'category_Ambiguous',
       'category_Imaginative Writing', 'category_Information Retrieval',
       'category_Opinion and Creativity', 'category_Other',
       'category_Problems in Specialized Areas', 'category_Seeking Advice',
       'category_Text Assistant',
       'emotion_is_potentially_emotional_dependence',
       'emotion_is_problematic_use', 'emotion_is_self_esteem',
       'emotion_is_emotional_vulnerability', 'emotion_is_loneliness',
       'emotion_expression_of_affection', 'emotion_expression_of_desire',
       'emotion_inquiry_into_personal_information',
       'emotion_persona

In [93]:
# drop 'first_input_polite_guard', 'intel_polite_label', 'intel_polite_score',
df_final = df_final.drop(columns=['first_input_polite_guard', 'intel_polite_label', 'intel_polite_score'])

# change is_polite to 4o_polite_judgement
df_final = df_final.rename(columns={'is_polite': '4o_polite_judgement'})

# change 'politeness_label','politeness_score' to "roberta_polite_judgement" and "roberta_polite_score"
df_final = df_final.rename(columns={'politeness_label': 'roberta_polite_judgement', 'politeness_score': 'roberta_polite_score'})

# change 'polite_label' and 'polite_score' to 'intel_polite_judgement' and 'intel_polite_score'
df_final = df_final.rename(columns={'polite_label': 'intel_polite_judgement', 'polite_score': 'intel_polite_score'})




In [97]:
df_final.head()

,conversation_hash,conversation,timestamp,turn,country,input_length,doable,first_user_input,is_english,is_gpt4,...,emotion_relationship_title,emotion_affectionate_language,emotion_attributing_human_qualities,emotion_non-normative_language,emotion_sentience,emotion_fear_of_addiction,first_turn_type,second_turn_type,intel_polite_judgement,intel_polite_score
0,fffee994232784af4f86e3e55396f1e5,[{'content': '17) by increasing the distance f...,2023-12-07 14:05:42+00:00,1,United States,257,1,,1,1,...,False,False,False,False,False,False,fact_checking,not_classified,neutral,0.935664
1,50008d5b54b2e0cf2c64bb27e9b313ea,"[{'content': '(A∈B, C, D)(B∈A, C, D)(C∈B, A, D...",2024-01-24 07:24:46+00:00,1,Russia,44,1,,0,0,...,False,False,False,False,False,False,no_question,not_classified,neutral,0.894242
2,52898c75e57246cf46345511981a73c8,[{'content': ' As ...,2024-04-05 06:49:58+00:00,1,China,3839,1,,1,0,...,False,False,False,False,False,False,no_question,not_classified,polite,0.691003
3,c16e41c4889260e2954984f00c917bde,"[{'content': '上游垂体刺激素直接作用MAFLD的最新研究进展', 'count...",2023-05-28 13:54:33+00:00,2,China,23,1,,0,0,...,False,False,False,False,False,False,no_question,no_question,neutral,0.417057
4,bc1f3b16ef16511cde79a240a69a7b49,[{'content': '不少人选择用Citywalk的方式去邂逅城市的独家记忆一切如同开...,2023-10-19 06:34:29+00:00,4,United States,503,1,,0,0,...,False,False,False,False,False,False,no_question,no_question,polite,0.878041


In [94]:
df_final.columns


Index(['conversation_hash', 'conversation', 'timestamp', 'turn', 'country',
       'input_length', 'doable', 'first_user_input', 'is_english', 'is_gpt4',
       'short_id', 'is_politics', 'is_social_need', 'is_fictional_roleplay',
       '4o_polite_judgement', 'special_char_ratio', 'punctuation_ratio',
       'avg_sent_length_words', 'dale_chall_readability',
       'roberta_polite_judgement', 'roberta_polite_score',
       'category_Ambiguous', 'category_Imaginative Writing',
       'category_Information Retrieval', 'category_Opinion and Creativity',
       'category_Other', 'category_Problems in Specialized Areas',
       'category_Seeking Advice', 'category_Text Assistant',
       'emotion_is_potentially_emotional_dependence',
       'emotion_is_problematic_use', 'emotion_is_self_esteem',
       'emotion_is_emotional_vulnerability', 'emotion_is_loneliness',
       'emotion_expression_of_affection', 'emotion_expression_of_desire',
       'emotion_inquiry_into_personal_information',
 

In [99]:
# describe intel_polite_judgement
df_final['intel_polite_judgement'].value_counts()




intel_polite_judgement
polite             4937
neutral            3749
impolite           1182
somewhat polite      61
Name: count, dtype: int64

In [96]:
# save df_final
df_final.to_parquet("wildchat_10k_full_enhanced.parquet")


In [100]:
df_final['roberta_polite_judgement'].value_counts()


roberta_polite_judgement
polite      5329
impolite    4602
Name: count, dtype: int64

In [102]:
# Find where both roberta and intel models consider the text impolite
impolite_df = df_final[(df_final['roberta_polite_judgement'] == 'impolite') & 
                       (df_final['intel_polite_judgement'] == 'impolite')]

# Display the count of rows where both models agree on impoliteness
print(f"Number of conversations both models consider impolite: {len(impolite_df)}")

# Look at a sample of these impolite conversations
if len(impolite_df) > 0:
    # Extract first user message from a sample conversation
    sample_idx = impolite_df.index[0]
    sample_convo = impolite_df.loc[sample_idx, 'conversation']
    first_msg = None
    
    # Try to extract the first user message
    try:
        if isinstance(sample_convo, str):
            import ast
            convo_list = ast.literal_eval(sample_convo)
        else:
            convo_list = sample_convo
            
        for msg in convo_list:
            if msg.get('role') == 'user':
                first_msg = msg.get('content')
                break
    except:
        first_msg = "Could not extract message"
    
    print("\nSample impolite first message:")
    print(first_msg[:500] + "..." if first_msg and len(first_msg) > 500 else first_msg)
    
    # Show politeness scores
    print(f"\nRoberta politeness score: {impolite_df.loc[sample_idx, 'roberta_polite_score']}")
    print(f"Intel politeness score: {impolite_df.loc[sample_idx, 'intel_polite_score']}")


Number of conversations both models consider impolite: 914

Sample impolite first message:
федеральная целевая программа отходы

Roberta politeness score: 0.8209155797958374
Intel politeness score: 0.4527910351753235


In [103]:
# Compare category distributions between impolite conversations and the full dataset

# First, let's get the category columns
category_columns = [col for col in df_final.columns if col.startswith('category_')]

# Create a function to display category distributions
def display_category_distribution(df, title):
    print(f"\n{title}:")
    
    # Count conversations in each category
    category_counts = {}
    for col in category_columns:
        # Sum the 1.0 values in each category column
        category_counts[col.replace('category_', '')] = df[col].sum()
    
    # Convert to Series for better display
    category_series = pd.Series(category_counts).sort_values(ascending=False)
    
    # Calculate percentages
    total = len(df)
    category_percentages = category_series / total * 100
    
    # Display counts and percentages
    for category, count in category_series.items():
        percentage = category_percentages[category]
        print(f"{category}: {int(count)} ({percentage:.2f}%)")
    
    return category_series

# Display distributions
all_categories = display_category_distribution(df_final, "Category Distribution in Full Dataset")
impolite_categories = display_category_distribution(impolite_df, "Category Distribution in Impolite Conversations")

# Calculate and display the relative difference
print("\nRelative Prevalence in Impolite vs Full Dataset:")
if len(impolite_df) > 0:
    # Calculate the percentage point difference
    full_dataset_pct = all_categories / len(df_final) * 100
    impolite_pct = impolite_categories / len(impolite_df) * 100
    
    # Calculate relative difference (how many times more/less prevalent)
    relative_diff = {}
    for category in all_categories.index:
        if category in impolite_pct.index and full_dataset_pct[category] > 0:
            relative_diff[category] = impolite_pct[category] / full_dataset_pct[category]
    
    # Display sorted by relative difference
    for category, ratio in sorted(relative_diff.items(), key=lambda x: x[1], reverse=True):
        direction = "more" if ratio > 1 else "less"
        print(f"{category}: {abs(ratio):.2f}x {direction} prevalent in impolite conversations")



Category Distribution in Full Dataset:
Imaginative Writing: 3007 (30.28%)
Text Assistant: 2309 (23.25%)
Problems in Specialized Areas: 1862 (18.75%)
Information Retrieval: 1263 (12.72%)
Ambiguous: 444 (4.47%)
Seeking Advice: 425 (4.28%)
Opinion and Creativity: 376 (3.79%)
Other: 245 (2.47%)

Category Distribution in Impolite Conversations:
Imaginative Writing: 318 (34.79%)
Information Retrieval: 196 (21.44%)
Opinion and Creativity: 119 (13.02%)
Text Assistant: 103 (11.27%)
Problems in Specialized Areas: 59 (6.46%)
Ambiguous: 52 (5.69%)
Seeking Advice: 42 (4.60%)
Other: 25 (2.74%)

Relative Prevalence in Impolite vs Full Dataset:
Opinion and Creativity: 3.44x more prevalent in impolite conversations
Information Retrieval: 1.69x more prevalent in impolite conversations
Ambiguous: 1.27x more prevalent in impolite conversations
Imaginative Writing: 1.15x more prevalent in impolite conversations
Other: 1.11x more prevalent in impolite conversations
Seeking Advice: 1.07x more prevalent in i

In [109]:
# Create a generator function to iterate through conversations in a specific category
def conversation_generator_by_category(df, category_name):
    """
    Generator function that yields conversations from a specific category one by one
    
    Args:
        df: DataFrame containing the conversations
        category_name: Name of the category to filter (without 'category_' prefix)
    
    Yields:
        tuple: (first_message, intel_judgment, roberta_judgment, conversation_hash)
    """
    # Filter conversations in the specified category
    category_col = f'category_{category_name}'
    if category_col not in df.columns:
        print(f"Category '{category_name}' not found in the dataset.")
        return
    
    category_df = df[df[category_col] == 1.0]
    
    if len(category_df) == 0:
        print(f"No conversations found in category '{category_name}'.")
        return
    
    # Iterate through each conversation in the category
    for idx, row in category_df.iterrows():
        try:
            # Extract the first user message from the conversation
            conversation = row['conversation']
            first_message = conversation[0]['content']
            
            # Get politeness judgments
            intel_judgment = row.get('intel_polite_judgement', 'N/A')
            roberta_judgment = row.get('roberta_polite_judgement', 'N/A')
            
            # Get conversation hash for reference
            conversation_hash = row.get('conversation_hash', 'N/A')
            
            yield (first_message, intel_judgment, roberta_judgment, conversation_hash)
        except (IndexError, KeyError, TypeError) as e:
            # Skip conversations with invalid format
            continue

# Function to display the next conversation from a generator
def display_next_conversation(generator):
    """
    Display the next conversation from a generator
    
    Args:
        generator: Generator function that yields conversations
    
    Returns:
        bool: True if a conversation was displayed, False if no more conversations
    """
    try:
        first_message, intel_judgment, roberta_judgment, conversation_hash = next(generator)
        
        print("-" * 80)
        print(f"Conversation Hash: {conversation_hash}")
        print(f"Intel Politeness: {intel_judgment} | Roberta Politeness: {roberta_judgment}")
        print("-" * 80)
        print(first_message[:500] + "..." if len(first_message) > 500 else first_message)
        print("-" * 80)
        return True
    except StopIteration:
        print("No more conversations in this category.")
        return False


# To get the next conversation, call display_next_conversation again:
# display_next_conversation(opinion_gen)  # Get next opinion conversation
# display_next_conversation(creativity_gen)  # Get next creativity conversation




In [111]:
creativity_gen = conversation_generator_by_category(df_final, "Opinion and Creativity")

# Display the first conversation from each category
print("\nFirst conversation in Imaginative Writing category:")
display_next_conversation(creativity_gen)



First conversation in Imaginative Writing category:
--------------------------------------------------------------------------------
Conversation Hash: c09e09014cad46185210dea43e3e1b33
Intel Politeness: polite | Roberta Politeness: impolite
--------------------------------------------------------------------------------
Based on the following information what groups could the term 'Sunni extremists' conceivably refer to here?


While U.S. troops policed Iraq to give its politicians space to reconstitute their state after Saddam, Iran grasped the opportunity to attack American forces, helping turn a transitional period into a bloody war (with ample help from Sunni extremists). Instead of building a new Iraq from scratch, America stabilized Iraq and quickly ran for the exits, putting a band-aid on the problem as ...
--------------------------------------------------------------------------------


True

In [156]:
display_next_conversation(creativity_gen)

--------------------------------------------------------------------------------
Conversation Hash: 15e23227f7d99e57928befa9c78ab3f6
Intel Politeness: impolite | Roberta Politeness: impolite
--------------------------------------------------------------------------------
What Would Southern English, Midlands English, And Northern English Versions of The Words "Yeah, I Know! She Was So Surprised!" Look Like?
--------------------------------------------------------------------------------


True

In [107]:

# Display examples from Opinion and Creativity categories
print("\n" + "="*40 + " OPINION CATEGORY EXAMPLES " + "="*40)
display_first_messages_by_category(df_final, "Opinion and Creativity", n=2)



======================================== OPINION CATEGORY EXAMPLES ========================================

First 2 user messages in 'Opinion and Creativity' category:
--------------------------------------------------------------------------------
Example 1:
Intel Politeness: polite | Roberta Politeness: impolite
Based on the following information what groups could the term 'Sunni extremists' conceivably refer to here?


While U.S. troops policed Iraq to give its politicians space to reconstitute their state after Saddam, Iran grasped the opportunity to attack American forces, helping turn a transitional period into a bloody war (with ample help from Sunni extremists). Instead of building a new Iraq from scratch, America stabilized Iraq and quickly ran for the exits, putting a band-aid on the problem as ...
--------------------------------------------------------------------------------
Example 2:
Intel Politeness: impolite | Roberta Politeness: impolite
Come up with 10 clever sayin

In [159]:
# Create a JSONL file with only the first user input for conversations where first_turn_type is not 'no_question'
import json

# Filter the dataframe to include only rows where first_turn_type is not 'no_question'
question_df = df_final[df_final['first_turn_type'] != 'no_question']

# Create a list to store the JSONL entries
jsonl_entries = []

# Iterate through the filtered dataframe
for _, row in question_df.iterrows():
    # Get the conversation hash
    conversation_hash = row['conversation_hash']
    
    # Get the conversation and clean it
    conversation = clean_conversation(row['conversation'])
    
    # Get the first user message from the conversation
    if isinstance(conversation, list) and len(conversation) > 0:
        first_user_message = next((msg for msg in conversation if msg.get('role') == 'user'), None)
        
        if first_user_message:
            # Create a new conversation list with only the first user message
            new_conversation = [first_user_message]
            
            # Create the JSONL entry
            entry = {
                "conversation_hash": conversation_hash,
                "conversation": new_conversation
            }
            
            jsonl_entries.append(entry)

# Write the entries to a JSONL file
output_file = "input_data/questions_first_turn_only.jsonl"
with open(output_file, 'w', encoding='utf-8') as f:
    for entry in jsonl_entries:
        f.write(json.dumps(entry) + '\n')

print(f"Created JSONL file with {len(jsonl_entries)} question conversations at: {output_file}")


Created JSONL file with 3604 question conversations at: input_data/questions_first_turn_only.jsonl


In [165]:
# Load the intent classifications from the JSONL file
import json
import pandas as pd
import os

# Path to the intent classifications file
intent_file = "output_data/intent_classifications.jsonl"

# Check if the file exists
if os.path.exists(intent_file):
    # Load the classifications
    intent_classifications = {}
    with open(intent_file, 'r', encoding='utf-8') as f:
        for line in f:
            data = json.loads(line)
            conversation_hash = data['conversation_hash']
            classifications = data['classifications']
            
            # Convert yes/no to boolean values
            intent_classifications[conversation_hash] = {
                'is_action_requesting': classifications['is_action_requesting'] == 'yes',
                'is_social_relational': classifications['is_social_relational'] == 'yes',
                'is_meta_conversational': classifications['is_meta_conversational'] == 'yes'
            }
    
    # Create a function to get classifications or default to False for a conversation hash
    def get_classification(hash_value):
        if hash_value in intent_classifications:
            return intent_classifications[hash_value]
        else:
            return {
                'is_action_requesting': False,
                'is_social_relational': False,
                'is_meta_conversational': False
            }
    
    # Add the classifications to the dataframe with renamed columns
    df_final['question_is_action_requesting'] = df_final['conversation_hash'].apply(
        lambda x: get_classification(x)['is_action_requesting'])
    df_final['question_is_social_relational'] = df_final['conversation_hash'].apply(
        lambda x: get_classification(x)['is_social_relational'])
    df_final['question_is_meta_conversational'] = df_final['conversation_hash'].apply(
        lambda x: get_classification(x)['is_meta_conversational'])
    
    # Display the count of each classification type
    print("Classification counts:")
    for col in ['question_is_action_requesting', 'question_is_social_relational', 'question_is_meta_conversational']:
        print(f"\n{col}:")
        print(df_final[col].value_counts())
    
    # Display sample rows with their classifications
    print("\nSample rows with classifications:")
    sample_cols = ['conversation_hash', 'question_is_action_requesting', 'question_is_social_relational', 'question_is_meta_conversational']
    print(df_final[sample_cols].sample(5))
else:
    print(f"Intent classifications file not found at: {intent_file}")



Classification counts:

question_is_action_requesting:
question_is_action_requesting
False    8349
True     1582
Name: count, dtype: int64

question_is_social_relational:
question_is_social_relational
False    9702
True      229
Name: count, dtype: int64

question_is_meta_conversational:
question_is_meta_conversational
False    9868
True       63
Name: count, dtype: int64

Sample rows with classifications:
                     conversation_hash  question_is_action_requesting  \
3112  6f7022f502126202d83e1107de0ced0b                          False   
1543  edf5e15e6ea59eb96381af509167a62f                          False   
4555  5a4610c25caaaab890337d19d2a06d8d                          False   
2090  b10b25f9ded7ff204bff741fb82b7c8b                          False   
2284  9c23d201971353eef3a1bbbd0c9b7dee                          False   

      question_is_social_relational  question_is_meta_conversational  
3112                          False                            False  
1543     

In [161]:
df_final.head()

,conversation_hash,conversation,timestamp,turn,country,input_length,doable,first_user_input,is_english,is_gpt4,...,emotion_non-normative_language,emotion_sentience,emotion_fear_of_addiction,first_turn_type,second_turn_type,intel_polite_judgement,intel_polite_score,is_action_requesting,is_social_relational,is_meta_conversational
0,fffee994232784af4f86e3e55396f1e5,[{'content': '17) by increasing the distance f...,2023-12-07 14:05:42+00:00,1,United States,257,1,,1,1,...,False,False,False,fact_checking,not_classified,neutral,0.935664,False,False,False
1,50008d5b54b2e0cf2c64bb27e9b313ea,"[{'content': '(A∈B, C, D)(B∈A, C, D)(C∈B, A, D...",2024-01-24 07:24:46+00:00,1,Russia,44,1,,0,0,...,False,False,False,no_question,not_classified,neutral,0.894242,not_classified,not_classified,not_classified
2,52898c75e57246cf46345511981a73c8,[{'content': ' As ...,2024-04-05 06:49:58+00:00,1,China,3839,1,,1,0,...,False,False,False,no_question,not_classified,polite,0.691003,not_classified,not_classified,not_classified
3,c16e41c4889260e2954984f00c917bde,"[{'content': '上游垂体刺激素直接作用MAFLD的最新研究进展', 'count...",2023-05-28 13:54:33+00:00,2,China,23,1,,0,0,...,False,False,False,no_question,no_question,neutral,0.417057,not_classified,not_classified,not_classified
4,bc1f3b16ef16511cde79a240a69a7b49,[{'content': '不少人选择用Citywalk的方式去邂逅城市的独家记忆一切如同开...,2023-10-19 06:34:29+00:00,4,United States,503,1,,0,0,...,False,False,False,no_question,no_question,polite,0.878041,not_classified,not_classified,not_classified


In [167]:
# save df_final
df_final.to_parquet("wildchat_10k_full_enhanced.parquet")

df_final.head()





,conversation_hash,conversation,timestamp,turn,country,input_length,doable,first_user_input,is_english,is_gpt4,...,emotion_non-normative_language,emotion_sentience,emotion_fear_of_addiction,first_turn_type,second_turn_type,intel_polite_judgement,intel_polite_score,question_is_action_requesting,question_is_social_relational,question_is_meta_conversational
0,fffee994232784af4f86e3e55396f1e5,[{'content': '17) by increasing the distance f...,2023-12-07 14:05:42+00:00,1,United States,257,1,,1,1,...,False,False,False,fact_checking,not_classified,neutral,0.935664,False,False,False
1,50008d5b54b2e0cf2c64bb27e9b313ea,"[{'content': '(A∈B, C, D)(B∈A, C, D)(C∈B, A, D...",2024-01-24 07:24:46+00:00,1,Russia,44,1,,0,0,...,False,False,False,no_question,not_classified,neutral,0.894242,False,False,False
2,52898c75e57246cf46345511981a73c8,[{'content': ' As ...,2024-04-05 06:49:58+00:00,1,China,3839,1,,1,0,...,False,False,False,no_question,not_classified,polite,0.691003,False,False,False
3,c16e41c4889260e2954984f00c917bde,"[{'content': '上游垂体刺激素直接作用MAFLD的最新研究进展', 'count...",2023-05-28 13:54:33+00:00,2,China,23,1,,0,0,...,False,False,False,no_question,no_question,neutral,0.417057,False,False,False
4,bc1f3b16ef16511cde79a240a69a7b49,[{'content': '不少人选择用Citywalk的方式去邂逅城市的独家记忆一切如同开...,2023-10-19 06:34:29+00:00,4,United States,503,1,,0,0,...,False,False,False,no_question,no_question,polite,0.878041,False,False,False
